In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.1,zoom_range=0.1,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Car Damage Dataset/body/training',
target_size = (256, 256),
batch_size = 10,
class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Car Damage Dataset/body/validation',
target_size = (256, 256),
batch_size = 10,
class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [4]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [5]:
vgg = VGG16(input_shape=[256,256] + [3], weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 5s 0us/step


In [6]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [7]:
# our Layers - you can add more if you want
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(3,activation='softmax')(x)

In [9]:
model1= Model(inputs=vgg.input, outputs=prediction)

In [10]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
import sys
r=model1.fit_generator(
training_set,
validation_data=test_set,
epochs=100,
steps_per_epoch=979//10,
validation_steps=171//10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/100
97/97 [==============================] - 20s 203ms/step - loss: 1.2468 - acc: 0.5418 - val_loss: 1.0699 - val_acc: 0.6412
Epoch 2/100
97/97 [==============================] - 19s 193ms/step - loss: 0.7592 - acc: 0.7441 - val_loss: 1.6222 - val_acc: 0.5765
Epoch 3/100
97/97 [==============================] - 19s 193ms/step - loss: 0.6435 - acc: 0.7946 - val_loss: 1.6233 - val_acc: 0.6059
Epoch 4/100
97/97 [==============================] - 19s 192ms/step - loss: 0.4110 - acc: 0.8524 - val_loss: 1.0407 - val_acc: 0.6412
Epoch 5/100
97/97 [==============================] - 19s 193ms/step - loss: 0.3594 - acc: 0.8679 - val_loss: 1.1046 - val_acc: 0.6412
Epoch 6/100
97/97 [==============================] - 19s 192ms/step - loss: 0.2769 - acc: 0.8999 - val_loss: 1.2588 - val_acc: 0.6353
Epoch 7/100
97/97 [==============================] - 19s 192ms/step - loss: 0.2382 - acc: 0.9226 - val_loss: 1.3195 - val_acc: 0.6941
Epoch 8/100
97/97 [==============================] - 19s 194ms

In [ ]:
model1.save('body.h5')

In [2]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [5]:
model=load_model('/content/drive/MyDrive/body.h5/')

In [6]:
def detect(frame):
    img = cv2.resize(frame,(256,256)) #resizing the image to model trained image size
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # uploaded image is in the form of BGR,so convert to RGB
    # scaling need to be done
    if(np.max(img)>1):
        img = img/255.0
    img = np.array([img]) #then to array fpormat
    prediction = model. predict (img)
    print(prediction)
    label = ["front","rear","side"]
    preds = label[np.argmax(prediction)]
    return preds

In [7]:
import numpy as np

In [8]:
data = '/content/drive/MyDrive/Colab Notebooks/damage-to-back-of-car.jpg'
image = cv2.imread(data)
print (detect(image))

1/1 [==============================] - 8s 8s/step
[[2.1170254e-06 9.9999785e-01 1.8356752e-22]]
rear
